In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 102.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [4]:
import os
import pandas as pd
import numpy as np
import shutil
import cv2
import yaml
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from tqdm import tqdm
import torch
import warnings
warnings.filterwarnings('ignore')


class DataPreparator:
    def __init__(self, csv_path, images_folder, output_folder='dataset_v1'):
        self.csv_path = csv_path
        self.images_folder = images_folder
        self.output_folder = output_folder
        self.classes = None
        self.class_to_idx = None

    def make_dirs(self):
        dirs = [
            f'{self.output_folder}/images/train',
            f'{self.output_folder}/images/test',
            f'{self.output_folder}/labels/train',
            f'{self.output_folder}/labels/test'
        ]
        for d in dirs:
            os.makedirs(d, exist_ok=True)

    def bbox_to_yolo(self, img_w, img_h, xmin, ymin, xmax, ymax):
        x_center = (xmin + xmax) / 2.0 / img_w
        y_center = (ymin + ymax) / 2.0 / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h
        return np.clip(x_center, 0, 1), np.clip(y_center, 0, 1), np.clip(width, 0, 1), np.clip(height, 0, 1)

    def prepare(self):
        df = pd.read_csv(self.csv_path, header=None,
                         names=['filename', 'class', 'xmin', 'ymin', 'xmax', 'ymax'])
        
        self.classes = sorted(df['class'].unique())
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        print(f"Total annotations: {len(df)}")
        print(f"Classes: {self.classes}")
        for cls in self.classes:
            count = len(df[df['class'] == cls])
            print(f"{cls}: {count} ({count/len(df)*100:.1f}%)")

        unique_images = df['filename'].unique()
        train_imgs, test_imgs = train_test_split(unique_images, test_size=0.2, random_state=42)
        
        print(f"\nTrain: {len(train_imgs)}, Test: {len(test_imgs)}")
        
        self.make_dirs()

        for split, img_list in [('train', train_imgs), ('test', test_imgs)]:
            for img_name in tqdm(img_list, desc=split):
                src = os.path.join(self.images_folder, img_name)
                if not os.path.exists(src):
                    continue
                
                dst = os.path.join(f'{self.output_folder}/images/{split}', img_name)
                shutil.copy(src, dst)
                
                img = cv2.imread(src)
                if img is None:
                    continue
                h, w = img.shape[:2]
                
                annotations = df[df['filename'] == img_name]
                label_file = os.path.join(f'{self.output_folder}/labels/{split}',
                                         img_name.replace('.jpg', '.txt').replace('.JPG', '.txt'))
                
                with open(label_file, 'w') as f:
                    for _, row in annotations.iterrows():
                        cls_idx = self.class_to_idx[row['class']]
                        xc, yc, bw, bh = self.bbox_to_yolo(w, h, row['xmin'], row['ymin'], 
                                                           row['xmax'], row['ymax'])
                        f.write(f"{cls_idx} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

        yaml_data = {
            'path': os.path.abspath(self.output_folder),
            'train': 'images/train',
            'val': 'images/test',
            'nc': len(self.classes),
            'names': self.classes
        }
        
        with open('config_v1.yaml', 'w') as f:
            yaml.dump(yaml_data, f)
        
        return self.classes


class ModelTrainer:
    def __init__(self, config_file='config_v1.yaml'):
        self.config = config_file
        self.model = None
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Device: {self.device}")

    def train_model(self, epochs=60, batch=16, img_size=640):
        self.model = YOLO('yolov8n.pt')
        
        batch_size = batch if self.device == 'cuda' else 8
        
        self.model.train(
            data=self.config,
            epochs=epochs,
            imgsz=img_size,
            batch=batch_size,
            device=self.device,
            workers=4 if self.device == 'cuda' else 2,
            name='model_v1',
            patience=15,
            save=True,
            plots=True,
            hsv_h=0.02,
            hsv_s=0.75,
            hsv_v=0.45,
            degrees=10,
            translate=0.15,
            scale=0.6,
            shear=3.0,
            flipud=0.0,
            fliplr=0.5,
            mosaic=1.0,
            optimizer='AdamW',
            lr0=0.001,
            lrf=0.01,
            momentum=0.9,
            weight_decay=0.0005,
            warmup_epochs=5,
            box=8.0,
            cls=0.6,
            dfl=1.8,
        )

    def evaluate_model(self):
        best_path = 'runs/detect/model_v1/weights/best.pt'
        if not os.path.exists(best_path):
            import glob
            paths = glob.glob('**/best.pt', recursive=True)
            if paths:
                best_path = paths[0]
        
        self.model = YOLO(best_path)
        metrics = self.model.val(data=self.config, device=self.device, plots=True)
        
        print("\n" + "="*60)
        print("METRICS")
        print("="*60)
        print(f"Precision: {metrics.box.mp:.4f}")
        print(f"Recall: {metrics.box.mr:.4f}")
        print(f"mAP50: {metrics.box.map50:.4f}")
        print(f"mAP50-95: {metrics.box.map:.4f}")
        
        if hasattr(metrics.box, 'ap_class_index'):
            with open(self.config, 'r') as f:
                cfg = yaml.safe_load(f)
                classes = cfg.get('names', [])
            
            print("\nPer-Class:")
            for i, idx in enumerate(metrics.box.ap_class_index):
                if i < len(classes):
                    print(f"\n{classes[idx]}:")
                    print(f"  P: {metrics.box.p[i]:.4f}")
                    print(f"  R: {metrics.box.r[i]:.4f}")
                    print(f"  mAP50: {metrics.box.ap50[i]:.4f}")
                    print(f"  mAP50-95: {metrics.box.ap[i]:.4f}")
        
        return metrics

    def predict(self, source_folder, conf=0.3):
        self.model.predict(
            source=source_folder,
            conf=conf,
            iou=0.5,
            save=True,
            save_txt=True,
            device=self.device,
            project='runs/detect',
            name='predictions_v1',
            exist_ok=True
        )

    def visualize(self, pred_folder='runs/detect/predictions_v1', n=10):
        imgs = [f for f in os.listdir(pred_folder) if f.endswith(('.jpg', '.png'))][:n]
        
        cols = min(5, len(imgs))
        rows = (len(imgs) + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(20, 4*rows))
        if rows * cols > 1:
            axes = axes.ravel()
        else:
            axes = [axes]
        
        for idx, img_name in enumerate(imgs):
            if idx >= len(axes):
                break
            img = cv2.imread(os.path.join(pred_folder, img_name))
            if img is not None:
                axes[idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                axes[idx].axis('off')
                axes[idx].set_title(img_name[:20], fontsize=7)
        
        for idx in range(len(imgs), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig('results_v1.png', dpi=150, bbox_inches='tight')
        plt.close()


def main():
    print("\n" + "="*60)
    print("VARIANT 1: YOLOv8n")
    print("="*60)
    
    CSV_PATH = '/kaggle/input/yolo-cv/data.csv'
    IMAGES_FOLDER = '/kaggle/input/final-images'
    
    if not os.path.exists(CSV_PATH):
        CSV_PATH = 'data.csv'
        IMAGES_FOLDER = 'images'
    
    prep = DataPreparator(CSV_PATH, IMAGES_FOLDER)
    prep.prepare()
    
    trainer = ModelTrainer()
    trainer.train_model(epochs=60, batch=16)
    
    metrics = trainer.evaluate_model()
    
    trainer.predict('dataset_v1/images/test')
    trainer.visualize()
    
    print("\nDone")


if __name__ == "__main__":
    main()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

VARIANT 1: YOLOv8n
Total annotations: 4888
Classes: ['Platelets', 'RBC', 'WBC']
Platelets: 361 (7.4%)
RBC: 4155 (85.0%)
WBC: 372 (7.6%)

Train: 291, Test: 73


test: 100%|██████████| 73/73 [00:00<00:00, 91.29it/s]


Device: cuda
Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=8.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.6, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=config_v1.yaml, degrees=10, deterministic=True, device=0, dfl=1.8, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.75, hsv_v=0.45, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.9, mosaic=1.0, multi_scale=False, name=model_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=15, perspective=0.0, plots=Tr